In [1]:
%%javascript
$('.nbp-app-bar').toggle()

<IPython.core.display.Javascript object>

In [2]:
import os
import re
import json
import random
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import itertools as it
import matplotlib.pyplot as plt

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Masking
from keras.layers import Convolution2D, Convolution1D, MaxPooling2D, MaxPooling1D, LSTM, TimeDistributed
from keras.losses import categorical_crossentropy
from keras.utils import np_utils
from keras.datasets import mnist

%matplotlib inline

/Users/evankranzler/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def subsets(iterable, minimum=0):
    for i in range(minimum, len(iterable) + 1):
        for x in it.combinations(iterable, i):
            yield x


def one_in_another(s1, s2):
    if s1 == s2:
        return True
    if len(s1) > len(s2):
        return one_in_another(s2, s1)
    for i in range(len(s2) + 1):
        for j in range(i):
            if s2.replace(s2[j:i], '') == s1:
                return True
    return False


class Chord(object):
    def __init__(self, root=None, intervals=set()):
        self.root = root
        self.intervals = intervals
        if root is not None:
            self.intervals.add(root)

    def __str__(self):
        return '(root: ' + str(self.root) + ' intervals: ' + ', '.join([str(x) for x in self.intervals]) + ')'

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return self.root == other.root and self.intervals == other.intervals

    def transpose(self, half_steps=1):
        if self.root == None:
            return self
        return Chord((self.root + half_steps) % 12, set((i + half_steps) % 12 for i in self.intervals))

    def normalize(self):
        if self.root is None:
            return self
        return self.transpose(-1 * self.root)

    def add_note(self, note):
        return Chord(self.root, self.intervals.union([note % 12]))

    def signature(self):
        return [1 * (i == self.root) for i in range(12)] + [1 * (i in self.intervals) for i in range(12)]

    @staticmethod
    def all_chords():
        for x in subsets(range(12), minimum=3):
            temp = set(x)
            for i in temp:
                yield Chord(i, temp)


notes = [s + q for q in 'b#' for s in 'ABCDEFG'] + [s for s in 'ABCDEFG']


def isolate_stem(chord_word):
    if 'Bass' in chord_word:
        return 'Bass'
    if '\\' in chord_word:
        return
    elif '/' in chord_word:
        temp = chord_word.split('/')[0]
        for n in notes:
            if n in temp:
                return temp.replace(n, '')
    else:
        for n in notes:
            if n in chord_word:
                return chord_word.replace(n, '')


def stematize(chord):
    for note in notes:
        if note in chord:
            return stematize(chord.replace(note, str(note_to_num(note))))
    return chord


def note_to_num(note):
    notedict = {'A': 9, 'B': 11, 'C': 0, 'D': 2, 'E': 4, 'F': 5, 'G': 7}
    try:
        number = notedict[note[0]]
    except:
        number = 0
    if '#' in note:
        number = (number + 1) % 12
    if 'b' in note:
        number = (number - 1) % 12
    return number


note_cycle = dict()
for note in notes:
    temp = []
    n = note_to_num(note)
    for note2 in notes:
        if (note_to_num(note2) - 1) % 12 == n:
            temp += [note2]
    note_cycle[note] = temp


def stretch_chords(chords, length):
    c_len = len(chords)
    if c_len == 0:
        return []
    if c_len == length:
        return chords
    if length % c_len == 0:
        m = length // c_len
        out = []
        for chord in chords:
            out += [chord] * m
        return out
    return stretch_chords(chords + ['/'], length)


def process_leadsheets():
    sheet_dict = dict()
    for path in ['./the-imaginary-book-part-1-A-M/', './the-imaginary-book-part-2-N-Z/']:
        for fn in os.listdir(path=path):
            if '.ls' in fn:
                temp = []
                with open(path + fn) as file:
                    for line in file.readlines():
                        if 'meter' in line:
                            meter = line
                            meter = meter.replace(')', '').replace('(meter', '')
                            meter = meter.split()
                            meter = int(meter[0])
                        if '|' in line:
                            temp += re.sub("[\(\[].*?[\)\]]", "", line.replace('\n', '')).replace(')', '').split('|')
                temp = [x.split() for x in temp]
                temp = [x for x in temp if x != []]
                while True:
                    loop = False
                    for t in temp:
                        if len(t) > meter:
                            meter *= 2
                            loop = True
                            break
                    if not loop:
                        break
                temp = [stretch_chords(x, meter) for x in temp]
                temp = sum(temp, [])
                temp2 = []
                for i, t in enumerate(temp):
                    if t == '/':
                        temp2 += [temp2[i - 1]]
                    else:
                        temp2 += [t]
                temp = ' '.join(temp2)
                sheet_dict[fn.replace('.ls', '')] = temp
    with open('sheets.json', 'w') as file:
        json.dump(sheet_dict, file)


new_chord_defs = (
    'C.............. same as CM\nCM............. C major (c e g)\nC2............. same as CMadd9\nC5............. C five (c g)\nC6............. same as CM6\nC69............ same as CM69\nC6#11.......... same as CM6#11\nC69#11......... same as CM69#11\nC6b5........... same as CM6#11\nCM13........... C major thirteen (c e g b d a)\nCM13#11........ C major thirteen sharp eleven (c e g b d f# a)\nCmaj13......... same as CM13\nCMaj13......... same as CM13\nCmaj13#11...... same as CM13#11\nCMaj13#11...... same as CM13#11\nCM6............ C major six (c e g a)\nCM6#11......... C major six sharp eleven (c e g a f#)\nCM6b5.......... same as CM6#11\nCM69#11........ C major six nine sharp eleven (c e g a d f#)\nCM69........... C major six nine (c e g a d)\nCM7#11......... C major seven sharp eleven (c e g b f#)\nCM7............ C major seven (c e g b)\nCmaj7.......... same as CM7\nCMaj7.......... same as CM7\nCmaj7#11....... same as CM7#11\nCMaj7#11....... same as CM7#11\nCM7add13....... C major seven add 13 (c e g a b d)\nCM7b5.......... C major seven flat five (c e gb b)\nCM7b6.......... C major seven flat six (c e g ab b)\nCM7b9.......... C major seven flat nine (c e g b db)\nCM9............ C major nine (c e g b d)\nCM9#11......... C major nine sharp eleven (c e g b d f#)\nCmaj9.......... same as CM9\nCMaj9.......... same as CM9\nCmaj9#11....... same as CM9#11\nCMaj9#11....... same as CM9#11\nCM9b5.......... C major nine flat five (c e gb b d)\nCMadd9......... C major add nine (c e g d)\nCMb5........... C major flat five (c e gb)\nCMb6........... C major flat six (c e ab)\nCadd2.......... same as CMadd9\nCadd9.......... same as CMadd9\nCadd9no3....... same as CMsus2\n\nMinor Chords\n\nCm#5........... C minor sharp five (c eb g#)\nCm+............ same as Cm#5\nCm............. C minor (c eb g)\nCm11#5......... C minor eleven sharp five (c eb ab bb d f)\nCm11........... C minor eleven (c eb g bb d f)\nCm11b5......... C minor eleven flat five (c eb bb gb d f)\nCm13........... C minor thirteen (c eb g bb d f a)\nCm6............ C minor six (c eb g a)\nCm69........... C minor six nine (c eb g a d)\nCm7#5.......... C minor seven sharp five (c eb ab bb)\nCm7............ C minor seven (c eb g bb)\nCm7b5.......... C minor seven flat five (c eb gb bb)\nCh............. same as Cm7b5 (h for \"half-diminished\")\nCm9#5.......... C minor nine sharp five (c eb ab bb d)\nCm9............ C minor nine (c eb g bb d)\nCm9b5.......... C minor nine flat five (c eb bb gb d)\nCmM7........... C minor major seven (c eb g b)\nCmM7b6......... C minor major seven flat six (c eb g ab b)\nCmM9........... C minor major nine (c eb g b d)\nCmadd9......... C minor add nine (c eb g d)\nCmb6........... C minor flat six (c eb ab)\nCmb6M7......... C minor flat six major 7 (c eb ab b)\nCmb6b9......... C minor flat six flat nine (c eb ab db)\n\nDiminished Chords\n\nCdim........... C diminished triad (c eb gb)\nCo............. same as Cdim\nCdim7.......... C diminished seventh (c eb gb a)\nCo7............ same as Cdim7\nCoM7........... C diminished major seventh (c eb gb b)\nCo7M7.......... C diminished seventh major seventh (c eb gb a b)\n\nAugmented Chords\n\nCM#5........... C major sharp five (c e g#)\nC+............. same as CM#5\nCaug........... same as CM#5\nC+7............ same as C7#5\nCM#5add9....... C major sharp five add 9 (c e g# d)\nCM7#5.......... C major seven sharp five (c e g# b)\nCM7+........... same as CM7#5\nCM9#5.......... C major nine sharp five (c e g# b d)\nC+add9......... same as CM#5add9\n\nDominant Chords\n\nC7............. C seven (c e g bb)\nC7#5........... C seven sharp five (c e g# bb)\nC7+............ same as C7#5\nCaug7.......... same as C7#5\nC7aug.......... same as C7#5\nC7#5#9......... C seven sharp five sharp nine (c e g# bb d#)\nC7alt.......... same as C7#5#9\nC7b13.......... C seven flat thirteen (c e g bb ab)\nC7b5#9......... same as C7#9#11\nC7b5........... C seven flat five (c e gb bb)\nC7b5b13........ same as C7#11b13\nC7b5b9......... same as C7b9#11\nC7b5b9b13...... same as C7b9#11b13\nC7b6........... C seven flat six (c e g ab bb)\nC7b9#11........ C seven flat nine sharp eleven (c e g bb db f#)\nC7b9#11b13..... C seven flat nine sharp eleven flat thirteen (c e g bb db f# ab)\nC7b9........... C seven flat nine (c e g bb db)\nC7b9b13#11..... C seven flat nine flat thirteen sharp eleven (c e g bb db f# ab)\nC7b9b13........ C seven flat nine flat thirteen (c e g bb db ab)\nC7no5.......... C seven no five (c e bb)\nC7#11.......... C seven sharp eleven (c e g bb f#)\nC7#11b13....... C seven sharp eleven flat thirteen (c e g bb f# ab)\nC7#5b9#11...... C seven sharp five flat nine sharp 11 (c e g# bb db f#)\nC7#5b9......... C seven sharp five flat nine (c e g# bb db)\nC7#9#11........ C seven sharp nine sharp eleven (c e g bb d# f#)\nC7#9#11b13..... C seven sharp nine sharp eleven flat thirteen (c e g bb d# f# ab)\nC7#9........... C seven sharp nine (c e g bb d#)\nC7#9b13........ C seven sharp nine flat thirteen (c e g bb d# ab)\n\nC9............. C nine (c e g bb d)\nC9#5........... C nine sharp five (c e g# bb d)\nC9+............ same as C9#5\nC9#11.......... C nine sharp eleven (c e g bb d f#)\nC9#11b13....... C nine sharp eleven flat thirteen (c e g bb d f# ab)\nC9#5#11........ C nine sharp five sharp eleven (c e g# bb d f#)\nC9b13.......... C nine flat thirteen (c e g bb d ab)\nC9b5........... C nine flat five (c e gb bb d)\nC9b5b13........ same as C9#11b13\nC9no5.......... C nine no five (c e bb d)\n\nC13#11......... C thirteen sharp eleven (c e g bb d f# a)\nC13#9#11....... C thirteen sharp nine sharp eleven (c e g bb d# f# a)\nC13#9.......... C thirteen sharp nine (c e g bb d# a)\nC13............ C thirteen (c e g bb d a)\nC13b5.......... C thirteen flat five (c e gb a bb)\nC13b9#11....... C thirteen flat nine sharp eleven (c e g bb db f# a)\nC13b9.......... C thirteen flat nine (c e g bb db a)\n\nSuspensions\n\nCMsus2......... C major sus two (c d g)\nCMsus4......... C major sus four (c f g)\nCsus2.......... same as CMsus2\nCsus24......... C sus two four (c d f g)\nCsus4.......... same as CMsus4\nCsus4add9...... same as Csus24\nCsusb9......... C sus flat nine (c db f g)\nC4............. C four (c f bb eb)\nCquartal....... same as C4\nC7b9b13sus4.... same as C7sus4b9b13\nC7b9sus........ same as C7susb9\nC7b9sus4....... same as C7sus4b9\nC7b9sus4....... same as C7susb9\nC7sus.......... same as C7sus4\nC7sus4......... C seven sus four (c f g bb)\nC7sus4b9....... C seven sus four flat nine (c f g bb db)\nC7sus4b9b13.... C seven sus four flat nine flat thirteen (c f g bb db ab)\nC7susb9........ C seven sus flat nine (c db f g bb)\nC9sus4......... C nine sus four (c f g bb d)\nC9sus.......... same as C9sus4\nC11............ C eleven (c e g bb d f)\nC13sus......... same as C13sus4\nC13sus4........ C thirteen sus four (c f g bb d a)\n\nMiscellaneous\n\nCBlues......... C Blues (c eb f gb g bb) (Use upper case to avoid confusion with Cb = C flat)\nCBass.......... C Bass (c) (Use upper case to avoid confusion with Cb = C flat)\n')
chord_defs = [[y for y in x.split('.') if y != ''] for x in new_chord_defs.split('\n') if '.' in x]
equivs = [[x[0][1:], x[1].replace(' same as ', '').replace(' (h for "half-diminished")', '')[1:]] for x in chord_defs if
          ' same as ' in x[1]]
equivs += [['mMaj7', 'mM7'], ['h7', 'm7b5'], ['mb5', 'dim']]


def make_into_intervals(chord_def):
    temp = re.findall('\(([^\)]+)\)', chord_def[1])[0]
    temp = [chord_def[0][1:], [note_to_num(x.capitalize()) for x in temp.split()]]
    return [temp[0], [1 * (i in temp[1]) for i in range(12)]]


equiv_dict = dict([make_into_intervals(x) for x in chord_defs if ' same as ' not in x[1]])
equiv_dict = {**equiv_dict, **dict([[x[0], equiv_dict[x[1]]] for x in equivs])}
equiv_dict['NC'] = [0] * 12


def process_chord(chord):
    if 'NC' in chord:
        return Chord()
    if 'Bass' in chord:
        for note in notes:
            if chord[:len(note)] == note:
                break
        return Chord(note_to_num(note), {note_to_num(note)})
    if '\\' in chord:
        [poly, rest] = chord.split('\\')
        poly = process_chord(poly)
    else:
        poly = None
        rest = chord
    if '/' in chord:
        [shape, root] = rest.split('/')
    else:
        shape = rest
        root = None
    for note in notes:
        if shape[:len(note)] == note:
            break
    stem = shape.replace(note, '')
    chord_obj = Chord(0, set([i for i in range(12) if equiv_dict[stem][i]]))
    chord_obj = chord_obj.transpose(note_to_num(note))
    if root is not None:
        chord_obj.root = note_to_num(root)
    if poly is not None:
        for n in poly.intervals:
            chord_obj = chord_obj.add_note(n)
    return chord_obj


def process_leadsheet(sheet):
    return [process_chord(x) for x in sheet.split()]


def all_transpositions(sheet):
    temp = process_leadsheet(sheet)
    return [[chord.transpose(i) for chord in temp] for i in range(12)]


def transpose_note(note):
    return note_cycle[note][-1]


def transpose_chord_symbol(chord, interval=1):
    if interval == 0:
        return chord
    elif interval == 1:
        if 'NC' in chord:
            return 'NC'
        if 'Bass' in chord:
            for note in notes:
                if chord[:len(note)] == note:
                    break
            return transpose_note(note) + 'Bass'
        if '\\' in chord:
            return '\\'.join([transpose_chord_symbol(x) for x in chord.split('\\')])
        if '/' in chord:
            return '/'.join([transpose_chord_symbol(x) for x in chord.split('/')])
        for note in notes:
            if chord[:len(note)] == note:
                break
        stem = chord.replace(note, '')
        return transpose_note(note) + stem
    elif 1 < interval < 12:
        return transpose_chord_symbol(transpose_chord_symbol(chord, interval=interval - 1))
    else:
        return transpose_chord_symbol(chord, interval=(interval % 12))


def seq_generator(training_data, seq_len):
    i = 0
    l = len(training_data)
    while True:
        if i + seq_len + 1 > l:
            i = 0
        yield training_data[i:i + seq_len]  # , training_data[i + 1:i + seq_len + 1]
        i += 1


def batch_generator(seq_gen, batch_size):
    while True:
        X_out = []
        y_out = []
        for i in range(batch_size + 1):
            temp = next(seq_gen)
            if i != batch_size:
                X_out += [temp]
            if i != 0:
                y_out += [temp]
        yield np.array(X_out), np.array(y_out)


def strat_gen(data, batch_size, seq_len):
    data_len = len(data)
    data_slices = data_len // batch_size
    staggers = [data[i * data_slices:] + data[:i * data_slices] for i in range(batch_size)]
    staggers = [it.cycle(x) for x in staggers]
    X = [[next(x) for _ in range(seq_len)] for x in staggers]
    while True:
        y = [x[1:] + [next(staggers[i])] for i, x in enumerate(X)]
        yield np.array(X), np.array(y)
        X = [x for x in y]

def filename_string(dt):
    return '_{}_{}_{}_{}'.format(dt.toordinal(), dt.hour, dt.minute, dt.second)

In [4]:
def build_model(seq_len, batches, dropout_value=0.5, is_stateful=False, LSTM_size=512, LSTM_count=3):
    model = Sequential()

    model.add(Masking(
        batch_input_shape=(batches, seq_len, 24),
        input_shape=(seq_len, 24)
    ))

    model.add(TimeDistributed(Dense(96, activation='relu')))
    model.add(Dropout(dropout_value))

    model.add(TimeDistributed(Dense(96, activation='relu')))
    model.add(Dropout(dropout_value))

    for _ in range(LSTM_count):
        model.add(LSTM(
            LSTM_size,
            return_sequences=True,
            recurrent_dropout=dropout_value,
            dropout=dropout_value,
            stateful=is_stateful,
            implementation=2,
#             unroll=True,
#             batch_input_shape=(batches, seq_len, LSTM_size)
        ))

    model.add(TimeDistributed(Dense(24, activation='sigmoid')))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [5]:
with open('sheets.json') as file:
    sheet_dict = json.load(file)
print('Sheets loaded')

Sheets loaded


In [6]:
every_sheet = []
for sheet in sheet_dict.values():
    every_sheet += all_transpositions(sheet)
print('Transpositions applied')

Transpositions applied


In [7]:
training_data = [[x.signature() for x in sheet] for sheet in every_sheet]
print('Training data processed')

Training data processed


In [8]:
random.shuffle(training_data)

In [9]:
out = []
for x in training_data:
    out += x
    out += [24 * [0]] * 8
print('Training data ready for usage')

Training data ready for usage


In [9]:
for name in os.listdir(path='./'):
    if '736788' in name:
        print(name)

model_not_stateful_2_epochs_736788_6_26_16
model_not_stateful_1_epochs_736788_1_14_33
model_not_stateful_3_epochs_736788_11_36_55


In [403]:
model = load_model('./model_not_stateful_1_epochs_736786_21_47_54')
# model = load_model('./model_not_stateful_1_epochs_736788_1_14_33')
# model = load_model('./model_not_stateful_2_epochs_736788_6_26_16')
# model = load_model('./model_not_stateful_3_epochs_736788_11_36_55')
# last_epoch = 5
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (128, 32, 24)             0         
_________________________________________________________________
time_distributed_1 (TimeDist (128, 32, 48)             1200      
_________________________________________________________________
dropout_1 (Dropout)          (128, 32, 48)             0         
_________________________________________________________________
time_distributed_2 (TimeDist (128, 32, 48)             2352      
_________________________________________________________________
dropout_2 (Dropout)          (128, 32, 48)             0         
_________________________________________________________________
lstm_1 (LSTM)                (128, 32, 512)            1148928   
_________________________________________________________________
lstm_2 (LSTM)                (128, 32, 512)            2099200   
__________

In [11]:
last_epoch = 0
epoch_stride = 1

seq_len = 32
is_stateful = False
dropout_value = 0.5
LSTM_size = 1024
LSTM_count = 3
if is_stateful:
    batches = 1
else:
    batches = 4 * seq_len

In [101]:
model = build_model(
    seq_len=seq_len,
    batches=batches,
    dropout_value=dropout_value,
    is_stateful=is_stateful,
    LSTM_size=LSTM_size,
    LSTM_count=LSTM_count
)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_28 (Masking)         (128, 32, 24)             0         
_________________________________________________________________
time_distributed_70 (TimeDis (128, 32, 96)             2400      
_________________________________________________________________
dropout_57 (Dropout)         (128, 32, 96)             0         
_________________________________________________________________
time_distributed_71 (TimeDis (128, 32, 96)             9312      
_________________________________________________________________
dropout_58 (Dropout)         (128, 32, 96)             0         
_________________________________________________________________
lstm_44 (LSTM)               (128, 32, 1024)           4591616   
_________________________________________________________________
lstm_45 (LSTM)               (128, 32, 1024)           8392704   
__________

In [12]:
# if is_stateful:
#     gen = strat_gen(out, batches, seq_len)
# else:
out = np.asarray(out)
gen = batch_generator(seq_generator(out, seq_len), batches)

In [103]:
history = model.fit_generator(
    gen,
    steps_per_epoch=(len(out) // batches),
    epochs=(last_epoch + epoch_stride),
    verbose=1,
    initial_epoch=last_epoch
)

last_epoch += epoch_stride

Epoch 1/1
  305/35159 [..............................] - ETA: 5:25:55 - loss: 0.4732 - acc: 0.7895

KeyboardInterrupt: 

In [ ]:
model_name = 'model_'
if is_stateful:
    model_name += 'is_stateful_'
else:
    model_name += 'not_stateful_'

right_now = datetime.datetime.now()

model_name += str(last_epoch)
model_name += '_epochs'
model_name += filename_string(right_now)

model.save(model_name)

In [392]:
chords = set()
for key in sheet_dict:
    temp = sheet_dict[key].split()
    for chord in temp:
        chords.add(chord)

chord_formats = set()
for x in chords:
    chord_formats.add(isolate_stem(x))

chord_objs = dict()
for chord in chords:
    temp = process_chord(chord)
    name = chord
    for _ in range(12):
        name = transpose_chord_symbol(name)
        temp = temp.transpose()
        chord_objs[name] = temp

canon_sigs = [np.array(chord_objs[x].signature()) for x in canonical_chords]

def remove_if_possible(list_in, element):
    out = []
    for d in list_in:
        cont = False
        for x in d:
            if element in x:
                cont = True
                break
        if cont:
            temp = [x for x in d if element not in x]
            if len(temp) > 0:
                out += [temp]
            else:
                out += [d]
        else:
            out += [d]
    return out

eclasses = []
for value in chord_objs.values():
    if value.root == 0:
        temp = []
        for key in chord_objs.keys():
            if chord_objs[key] == value:
                temp += [key]
        if temp not in eclasses:
            eclasses += [temp]

            
decisions = [x for x in eclasses if len(x) > 1]

stuff = remove_if_possible(decisions, 'maj')

out2 = []
for d in stuff:
    cont = False
    for x in d:
        if 'sus' in x:
            cont = True
            break
    if cont:
        temp = [x for x in d if 'sus' not in x or 'sus4' in x or 'sus2' in x]
        if len(temp) > 0:
            out2 += [temp]
        else:
            out2 += [d]
    else:
        out2 += [d]

out3 = remove_if_possible(out2, '\\')
out4 = remove_if_possible(out3, '6')
out5 = remove_if_possible(out4, '/')

out6 = []
for x in out5:
    if len(x) == 2 and one_in_another(*x):
        out6 += [[min(x,key=len)]]
    else:
        out6 += [x]
        
decided = [x[0] for x in out6 + [y for y in eclasses if len(y) > 0]] + ['NC']

canonical_chords = []
for d in decided:
    for i in range(12):
        temp = transpose_chord_symbol(d, i)
        if temp not in canonical_chords:
            canonical_chords += [temp]

reverse_dict = dict()
for chord in canonical_chords:
    temp = chord_objs[chord].signature()
    reverse_dict[''.join([str(x) for x in temp])] = chord

def binary_crossentropy(y_true, y_predict):
    return -sum([(y1 * np.log(y2)) + ((1 - y1) * np.log(1 - y2)) for y1, y2 in zip(y_true, y_predict)])

def hamming(s1, s2):
    assert(len(s1) == len(s2))
    return sum([x != y for x, y in zip(s1, s2)])

def sample_chord(chord):
    prob_array = np.array([1 / binary_crossentropy(x, c) for x in canon_sigs])
    prob_array = prob_array / prob_array.sum()
    return canon_sigs[np.random.choice(np.arange(len(canon_sigs)),p=prob_array)]

def guess_chord(chord):
    root = np.random.choice(np.arange(12), p=(chord[:12] / sum(chord[:12])))
    temp = 1 * (np.array(chord[12:]) > np.random.random(12))
    return np.array([1 * (i == root) for i in range(12)] + [1 if temp[i] == 1 or i == root else 0 for i in range(12)])

def process_prediction(predix):
    return [1 * (i == predix[:12].argmax() and max(predix) > .5) for i in range(12)] + [1 * (x > 0.5) for x in predix[12:]]

def turn_to_word(chord):
    temp = [str(x) for x in (chord)]
    for i in range(12):
        if temp[i] == '1':
            temp[i + 12] = '1'
    temp = ''.join(temp)
    try:
        symbol = reverse_dict[temp]
    except:
        symbol = min([[np.linalg.norm(np.array(chord_objs[x].signature()) - chord), x] for x in canonical_chords])
#         symbol2 = reverse_dict[min([[hamming(temp, key), key] for key in reverse_dict.keys()])[1]]
        print('mismatch:', temp, symbol[1], symbol[0])
        symbol = symbol[1]
    return symbol

def eightify(stuff):
#     assert(len(stuff) == 32)
    out = ''
    for i in range(len(stuff)):
        if i % 8 == 0 and i != 0:
            out += '\n'
        out += stuff[i]
        out += ' | '
    return out

In [404]:
gen1 = seq_generator(out, seq_len)
# [next(gen1) for _ in range(128)]
t = next(gen1)
T = np.stack([t] * batches)

In [405]:
chords_to_make = 64
for i in range(chords_to_make):
    predix = model.predict(T[:,-32:,:], batch_size=batches)
    predix = np.expand_dims(predix[:,-1], axis=1)
    predix = np.array([[sample_chord(predix[0][0])]] * batches)
    T = np.concatenate([T, predix], axis=1)
    t = next(gen1)
    print(round((100 * i + 100) / chords_to_make, 2), end='%           \r')

In [406]:
thing = eightify([turn_to_word(y) for y in T[0][32:]])

In [407]:
print(thing)
print()
print(eightify([turn_to_word(y) for y in np.array(t)]))

Co7M7 | G7sus | G#7/D | C/F# | B7#5b9/E | Bm6 | G#M7#11/D# | EM7/G# | 
EMb5 | Amaj7 | E6/G# | G#7+ | G#M7#5/G | A#7b9/D# | G#7b9/C# | D7sus | 
Am9/G | B/A | BM9#11 | C#7sus4/G# | F#m7/A# | D#7sus | C#9 | A69/C# | 
D#7/A# | C#o/E | C#o/G# | G#m9/C# | F#7alt | G9b5 | C#7#5#9/F | A11 | 
C#/F# | Em9/A | A | A#5 | G13b9#11 | Adim | BM9#11 | Fm7/A | 
Am69 | G#13b5 | DM7/F | E7#9/B | D#7b6 | F7/C | F#o/A | Dm11b5 | 
D#m/D | CM6 | FM7/B | C#7#11 | D#7b9#11 | D#7b6/A# | G#7b9 | G9/F# | 
AM7b5/G# | A7sus4/E | A\F | AM7#5/E | EM9/B | F#6/A# | EM7#5/C | Bm/E | 

A7 | A7 | NC | NC | NC | NC | NC | NC | 
NC | NC | A#m | A#m | A#m | A#m | D#m | D#m | 
D#m | D#m | A#m | A#m | A#m | A#m | D#7 | D#7 | 
D#7 | D#7 | C# | C# | Bo7/D | Bo7/D | D#m7 | D#m7 | 


In [124]:
def dupe_counter(pred):
#     dupes = []
    return len(set([turn_to_word(x) for x in pred]))
#         if x not in dupes:
#             dupes += [x]
#     return (dupes)

In [78]:
gen1 = seq_generator(out, seq_len)
dupes = []
l = len(out) // 32
for i in range(l):
    p = round((100 * i) / l, 2)
    dupes += [dupe_counter(next(gen1))]
    [next(gen1) for _ in range(31)]
    print(p, end='       \r')

In [167]:
gen1 = seq_generator(out, seq_len)
dupes2 = []
# l = len(out) // 32
l = 1000
t = next(gen1)
for i in range(l):
    T = np.stack([t] * batches)
    p = round((100 * i) / l, 2)
    for _ in range(64):
        predix = model.predict(T[:,-32:,:], batch_size=batches)
        predix = np.expand_dims(predix[:,-1], axis=1)
        T = np.concatenate([T, predix], axis=1)
    [next(gen1) for _ in range(31)]
    t = next(gen1)
    dupes2 += [dupe_counter(T[0][64:])]
#     print(([turn_to_word(x) for x in T[0][64:]]))
    print(p, round(np.mean(dupes2), 2), round(np.std(dupes2), 2), end='       \r')
#     break

mismatch: 000000000001000000100101 B7 1.1018787761197362
mismatch: 000000000001000000100101 B5 1.0568486048282173
mismatch: 000000100000010010100000 F#7 0.980127709639786
mismatch: 000000100000000000100001 B5 1.2353488807056856
mismatch: 000000100000010000100101 B9/F# 1.1228914932501608
mismatch: 000000000001000000100101 B7sus4 1.3497807502163373
mismatch: 000000100000010010100000 F#7 0.847338682131144
mismatch: 000000100000010010100000 F#7 0.9294703115272414
mismatch: 000000100000010010100101 F#m7 0.7716156006973176
mismatch: 000000100000010000100101 F#m 1.034712632712423
mismatch: 000000000001000000100101 F#m/B 1.2150483420933096
mismatch: 000000100000000010100100 F#m7 1.0214412184902102
mismatch: 000000100000000010100100 F#m7 1.0016301368839198
mismatch: 000001000000001001001100 F13#9 1.0017180631207045
mismatch: 000000010000101001010001 G7 0.9416414875987901
mismatch: 000000000100100000010100 Am7b5 1.2433754737778897
mismatch: 000000100000000000100101 Esus4/F# 1.3369455231974399
mi

KeyboardInterrupt: 

In [222]:
binary_crossentropy(canon_sigs[0], c)

33.02319203812074